In [1]:
# HOG usage

# skimage.feature.hog(image, orientations=9, pixels_per_cell=(8, 8),
# cells_per_block=(3, 3), block_norm='L1', visualize=False, visualise=None, 
# transform_sqrt=False, feature_vector=True)

# orientations = 9
# pixels_per_cell = (8,8)
# cells_per_block = (2,2)

# if visualize == True:
#     return feature_vector, hog_image
# else:
#     return feature_vector

In [2]:
# glob usage

# glob.glob(pathname, recursive=False)

In [3]:
# some cv2 module constants

# cv2.COLOR_RGB2HSV = 41
# cv2.COLOR_RGB2LUV = 51
# cv2.COLOR_RGB2HLS = 53
# cv2.COLOR_RGB2YUV = 83
# cv2.COLOR_RGB2YCrCb = 37
# cv2.COLOR_RGB2GRAY = 7

In [47]:
import numpy as np
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob
import os
import random
from skimage.feature import hog
import csv
import time
import pickle

from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn import decomposition

In [48]:
# training image pathnames for glob
car_pathname = "training/vehicles/**/*.png"
non_car_pathname = "training/non-vehicles/**/*.png"

In [49]:
# extract pathnames of training images
cars = glob.glob(car_pathname, recursive=True)
non_cars = glob.glob(non_car_pathname, recursive=True)

print("len[cars]     = ",len(cars))
print("len[non_cars] = ", len(non_cars))

len[cars]     =  8792
len[non_cars] =  8968


In [50]:
# a function to convert color space from RGB to cspace

# cspace = HSV, LUV, HLS, YUV, YCrCb, GRAY

def convert_color(img, cspace):
    image = np.copy(img)
    
    constant_dict = {'RGB':cv2.COLOR_BGR2RGB, 
                     'HSV':cv2.COLOR_BGR2HSV, 
                     'LUV':cv2.COLOR_BGR2LUV,
                     'HLS':cv2.COLOR_BGR2HLS,
                     'YUV':cv2.COLOR_BGR2YUV,
                     'YCrCb':cv2.COLOR_BGR2YCR_CB,
                     'GRAY':cv2.COLOR_BGR2GRAY, 
                     'RGB2YCrCb':cv2.COLOR_RGB2YCR_CB, 
                     'BGR':cv2.COLOR_RGB2BGR}
    
    converted = cv2.cvtColor(image, constant_dict[cspace])
    
    return converted

In [37]:
# short version of sklearn.feature.hog()
def hog_short(img, vis=False, feature_vector=True):
    if vis is True:
        features, hog_image = hog(img, orientations=orient, cells_per_block=cell_size, 
                                  pixels_per_cell=pixels, transform_sqrt=True, visualise=True, 
                                 feature_vector=feature_vector)
        return features, hog_image
    
    else:
        features = hog(img, orientations=orient, cells_per_block=cell_size, 
                       pixels_per_cell=pixels, transform_sqrt=True, feature_vector=feature_vector)
        return features

In [38]:
# a function to iterate through training examples
 
# it takes pathnames as a list and returns hog feature vectors also as a list
# hog_channel = 0,1,2 or ALL

def hog_from_file(pathnames, hog_channel=[0,1,2], cspace='YCrCb', feature_vector=True):
    # Create a list to append feature vectors to
    all_features = []
    
    for image_path in pathnames:
        # read images one by one
        image = cv2.imread(image_path)
        # convert color space to desired one
        feat_img = convert_color(image, cspace)
            
        # extract features from single image
        features = []

        for channel in hog_channel:
            features.append(hog_short(feat_img[:,:,channel], feature_vector=feature_vector))

        features = np.ravel(features)

        # append features to return list
        all_features.append(features)
        
    return all_features

In [39]:
# takes pathnames of images
def hog_from_file_gray(pathnames, feature_vector=True):
    all_features = []
    
    for image_path in pathnames:
        # read images one by one
        image = cv2.imread(image_path)
        feat_img = convert_color(image, 'GRAY')
        
        features = hog_short(feat_img)

        features = np.ravel(features)

        # append features to return list
        all_features.append(features)
        
    return all_features

In [40]:
# takes images as a np.array / list of np.array's
def hog_from_list(imgs, hog_channel=[0,1,2], cspace='YCrCb', feature_vector=True):
    all_features = []
    
    for img in imgs:
        image = np.copy(img)
        
        # convert color space to desired one
        feat_img = convert_color(image, cspace)
            
        # extract features from single image
        
        features = []

        for channel in hog_channel:
            features.append(hog_short(feat_img[:,:,channel], feature_vector=feature_vector))

        features = np.ravel(features)

        # append features to return list
        all_features.append(features)
        
    return all_features

In [41]:
# takes images as a np.array / list of np.array's
def hog_from_list_gray(imgs, feature_vector=True):
    all_features = []
    
    for img in imgs:
        feat_img = np.copy(img)
        
        features = hog_short(feat_img)

        features = np.ravel(features)

        # append features to return list
        all_features.append(features)
        
    return all_features

In [44]:
def trainSVC(pos_examples, neg_examples, hog_channel=[0,1,2], cspace='GRAY', savename='_'):
    # time stamp before feature extraction
    t1 = time.time()

    # extracting HOG features of cars and non-cars
    print('SVM: using {} positive and {} negative examples'.format(len(pos_examples), len(neg_examples)))
    print('SVM: extracting features...')
    if cspace is 'GRAY':
        pos_features = hog_from_file_gray(pos_examples)
        neg_features = hog_from_file_gray(neg_examples)

    else:
        pos_features = hog_from_file(pos_examples, hog_channel=hog_channel, cspace=cspace)
        neg_features = hog_from_file(neg_examples, hog_channel=hog_channel, cspace=cspace)

    # feature vectors stack
    X = np.vstack((pos_features, neg_features)).astype(np.float64)
    # feature normalize
    print('SVM: normalizing features...')
    X_scaler = StandardScaler().fit(X)
    # apply normalization to X
    scaled_X = X_scaler.transform(X)
    
    # reduce features with pca
    pca = decomposition.PCA(n_components=1000)
    pca.fit(scaled_X)
    scaled_X = pca.transform(scaled_X)


    # labels vector
    y = np.hstack((np.ones(len(pos_features)), np.zeros(len(neg_features))))
    
    # time stamp after feature extraction
    t2 = time.time()
    print('{} seconds to create {} feature vectors of size {}'.format(round(t2-t1, 5), len(scaled_X), len(scaled_X[0])))

    
    
    #--------------------------------------------------------------------------------------------------#
    
    
    
    # split training and test sets randomly
    print('SVM: splitting train/validation data...')
    rand_state = np.random.randint(0,100)
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.1, random_state=rand_state)

    # create linear svc object
    print('SVM: Training model...')
    svc = LinearSVC()

    # create time stamps before and after training
    t1 = time.time()
    svc.fit(X_train, y_train)
    t2 = time.time()
    print(round(t2-t1, 2), 'Seconds to train SVM...')
    print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
    
    print('SVM: Saving model...')
    joblib.dump(svc, 'svm_'+savename+'.pkl')
    print('svm'+savename+'.pkl saved.')
    joblib.dump(X_scaler, 'svm_scaler_'+savename+'.pkl')
    print('svm_scaler_'+savename+'.pkl saved.')
    joblib.dump(pca, 'svm_pca_'+savename+'.pkl')
    print('svm_pca_'+savename+'.pkl saved.')
    print('Done.')
    
    return svc, X_scaler, X_test, y_test

In [45]:
svc, scaler_X, X_test, y_test = trainSVC(cars, non_cars, cspace='GRAY',savename='gtest')

SVM: using 8792 positive and 8968 negative examples
SVM: extracting features...


C:\Users\ms\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


SVM: normalizing features...
102.31971 seconds to create 17760 feature vectors of size 1000
SVM: splitting train/validation data...
SVM: Training model...
6.79 Seconds to train SVM...
Test Accuracy of SVC =  0.9465
SVM: Saving model...
svmgtest.pkl saved.
svm_scaler_gtest.pkl saved.
svm_pca_gtest.pkl saved.
Done.


In [46]:
# check the score of SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# check the prediction time for a single sample
t=time.time()
numb_predict = 15
print('SVC predicts: ', svc.predict(X_test[0:numb_predict]))
print('Labels        ', y_test[0:numb_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', numb_predict,'labels with SVC')

Test Accuracy of SVC =  0.9465
SVC predicts:  [ 1.  0.  1.  0.  1.  0.  0.  1.  0.  1.  1.  0.  0.  0.  0.]
Labels         [ 1.  0.  1.  0.  1.  0.  0.  1.  0.  1.  1.  0.  0.  0.  0.]
0.0 Seconds to predict 15 labels with SVC


In [ ]:
test = cv2.imread()